In [11]:
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from tqdm import tqdm

In [13]:
sablayan_clustered_df = pd.read_csv('Sablayan_Data_Clustered.csv')
sablayan_clustered_df

,Unnamed: 0,id,BARANGAY,PSGC,LAND COVERAGE,RHU,POPULATION DENSITY PER HEXAGON,POIs PER HEXAGON,ROADS,STORM SURGE HAZARD LEVEL,FLOOD HAZARD LEVEL,LANDSLIDE HAZARD LEVEL,LANDCOVER NUMERICAL,HCFAI,Cluster,Unnamed: 15
0,0.0,22.0,Ligaya,PH175109009,Brush/Shrubs,0.0,0.000000,1.0,1.0,0.0,0.0,2.0,1.0,0.000000,LOW,0
1,10.0,23.0,Burgos,PH175109009,Brush/Shrubs,0.0,0.000000,0.0,0.0,0.0,0.0,2.0,1.0,0.000000,LOW,0
2,20.0,49.0,Pag-Asa,PH175109004,NaN,0.0,154.538807,0.0,0.0,3.0,2.0,0.0,0.0,1151.197170,MID,1
3,21.0,50.0,San Agustin,PH175109004,Built Up,0.0,214.812672,0.0,0.0,3.0,2.0,0.0,0.0,1600.191852,MID,1
4,22.0,51.0,San Agustin,PH175109004,Grassland,0.0,302.902705,0.0,0.0,3.0,1.0,0.0,1.0,2469.775420,HIGH,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968,1060.0,1363.0,Pag-Asa,PH175109003,Brush/Shrubs,0.0,16.884621,0.0,0.0,0.0,0.0,3.0,1.0,235.553869,LOW,0
969,1061.0,1364.0,Pag-Asa,PH175109003,NaN,0.0,21.536779,0.0,0.0,0.0,0.0,3.0,0.0,293.994123,LOW,0
970,1062.0,1397.0,Pag-Asa,PH175109003,NaN,0.0,0.000000,0.0,0.0,0.0,0.0,3.0,0.0,0.000000,LOW,0
971,1072.0,1398.0,Pag-Asa,PH175109003,NaN,0.0,0.000000,0.0,0.0,0.0,0.0,3.0,0.0,0.000000,LOW,0


In [14]:
edited_df = sablayan_clustered_df.drop(['id', 'BARANGAY', 'PSGC', 'LAND COVERAGE', 'Cluster', 'Unnamed: 0'], axis=1)

In [15]:
edited_df

,RHU,POPULATION DENSITY PER HEXAGON,POIs PER HEXAGON,ROADS,STORM SURGE HAZARD LEVEL,FLOOD HAZARD LEVEL,LANDSLIDE HAZARD LEVEL,LANDCOVER NUMERICAL,HCFAI,Unnamed: 15
0,0.0,0.000000,1.0,1.0,0.0,0.0,2.0,1.0,0.000000,0
1,0.0,0.000000,0.0,0.0,0.0,0.0,2.0,1.0,0.000000,0
2,0.0,154.538807,0.0,0.0,3.0,2.0,0.0,0.0,1151.197170,1
3,0.0,214.812672,0.0,0.0,3.0,2.0,0.0,0.0,1600.191852,1
4,0.0,302.902705,0.0,0.0,3.0,1.0,0.0,1.0,2469.775420,2
...,...,...,...,...,...,...,...,...,...,...
968,0.0,16.884621,0.0,0.0,0.0,0.0,3.0,1.0,235.553869,0
969,0.0,21.536779,0.0,0.0,0.0,0.0,3.0,0.0,293.994123,0
970,0.0,0.000000,0.0,0.0,0.0,0.0,3.0,0.0,0.000000,0
971,0.0,0.000000,0.0,0.0,0.0,0.0,3.0,0.0,0.000000,0


In [16]:
edited_df.rename(columns = {'Unnamed: 15':'Cluster'}, inplace = True)

In [18]:
x = edited_df
x_normalized = (x - x.min()) / (x.max() - x.min())
x_normalized

,RHU,POPULATION DENSITY PER HEXAGON,POIs PER HEXAGON,ROADS,STORM SURGE HAZARD LEVEL,FLOOD HAZARD LEVEL,LANDSLIDE HAZARD LEVEL,LANDCOVER NUMERICAL,HCFAI,Cluster
0,0.0,0.000000,0.021277,1.0,0.0,0.000000,0.666667,1.0,0.000000,0.0
1,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.666667,1.0,0.000000,0.0
2,0.0,0.087617,0.000000,0.0,1.0,0.666667,0.000000,0.0,0.076793,0.5
3,0.0,0.121790,0.000000,0.0,1.0,0.666667,0.000000,0.0,0.106744,0.5
4,0.0,0.171734,0.000000,0.0,1.0,0.333333,0.000000,1.0,0.164751,1.0
...,...,...,...,...,...,...,...,...,...,...
968,0.0,0.009573,0.000000,0.0,0.0,0.000000,1.000000,1.0,0.015713,0.0
969,0.0,0.012210,0.000000,0.0,0.0,0.000000,1.000000,0.0,0.019611,0.0
970,0.0,0.000000,0.000000,0.0,0.0,0.000000,1.000000,0.0,0.000000,0.0
971,0.0,0.000000,0.000000,0.0,0.0,0.000000,1.000000,0.0,0.000000,0.0
